<a href="https://colab.research.google.com/github/peeti-sriwongsanguan/DrICE-TK/blob/main/dr_ice_tiktok_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd drive/MyDrive/Github/

/content/drive/MyDrive/Github


In [9]:
mkdir DrICE-TK

https://github.com/peeti-sriwongsanguan/DrICE-TK.git

In [73]:
!git init
!git config --global init.defaultBranch main
!git branch -m main

Reinitialized existing Git repository in /content/drive/MyDrive/Github/DrICE-TK/.git/


In [75]:
!git init

Reinitialized existing Git repository in /content/drive/MyDrive/Github/DrICE-TK/.git/


In [76]:
!git status

On branch main

No commits yet

nothing to commit (create/copy files and use "git add" to track)


In [77]:
!git add .

In [82]:
!git commit -m "Initial commit"
!git add .

On branch main

Initial commit

nothing to commit (create/copy files and use "git add" to track)


In [83]:
!git push -u origin main

error: src refspec main does not match any
error: failed to push some refs to 'origin'


In [79]:
!git config --global user.email "peeti@berkeley.edu"
!git config --global user.name "peeti"

In [81]:
!git branch

In [14]:
ls -a DrICE-TK/

.git/


In [19]:
%cd DrICE-TK

/content/drive/MyDrive/Github/DrICE-TK


In [20]:
!git remote add main https://github.com/peeti-sriwongsanguan/DrICE-TK.git

In [21]:
!git remote -v

main	https://github.com/peeti-sriwongsanguan/DrICE-TK.git (fetch)
main	https://github.com/peeti-sriwongsanguan/DrICE-TK.git (push)


In [22]:
!git status

On branch master

No commits yet

nothing to commit (create/copy files and use "git add" to track)


# for Thai language

In [42]:
!pip install pythainlp scikit-learn

In [43]:
import pandas as pd
import pythainlp
from pythainlp.corpus.common import thai_stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
import numpy as np

pd.set_option('display.max_columns', None)

In [34]:
df = pd.read_excel('../../data/socialMedia_sample.xlsx')
df.head()

/usr/local/lib/python3.10/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Date_submit,Platform,account_nm,ClipID,Name,No_Viewer,clip_lenght,avg_watch_time,%_of_engagement,%_gender (Men / Woman),Unnamed: 10,age_range
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Men,Woman,NaN
1,16/11/66,Tiktok,Dr.Ice.Official,10001.0,กินคอลลาเจนให้เห็นผลใน 7 วัน,1.1M,0.50,13.2,0.0995,18,80,18 - 55+
2,16/11/66,Tiktok,Dr.Ice.Official,10002.0,บูสผิวหน้าในหนึ่งคืนให้ทันเดท,8761,0.45,11.6,0.0736,22,77,18 - 55+
3,16/11/66,Tiktok,Dr.Ice.Official,10003.0,ถ้าหมอเลือกกินวิตามินได้ตัวเดียวจะกินอะไร,825.4k,0.47,15.2,0.1153,26,72,18 - 55+
4,17/11/66,Tiktok,Dr.Ice.Official,10004.0,วิธีจัดการสิวแบบเร่งด่วน,208.3k,0.55,17.3,0.1094,30,70,18 - 55+


In [37]:
df.rename(columns = {'%_gender (Men / Woman)':'male',
                     'Unnamed: 10':'female'}, inplace = True)
df = df.iloc[1:,:]
df.head()

,Date_submit,Platform,account_nm,ClipID,Name,No_Viewer,clip_lenght,avg_watch_time,%_of_engagement,male,female,age_range
1,16/11/66,Tiktok,Dr.Ice.Official,10001.0,กินคอลลาเจนให้เห็นผลใน 7 วัน,1.1M,0.50,13.2,0.0995,18,80,18 - 55+
2,16/11/66,Tiktok,Dr.Ice.Official,10002.0,บูสผิวหน้าในหนึ่งคืนให้ทันเดท,8761,0.45,11.6,0.0736,22,77,18 - 55+
3,16/11/66,Tiktok,Dr.Ice.Official,10003.0,ถ้าหมอเลือกกินวิตามินได้ตัวเดียวจะกินอะไร,825.4k,0.47,15.2,0.1153,26,72,18 - 55+
4,17/11/66,Tiktok,Dr.Ice.Official,10004.0,วิธีจัดการสิวแบบเร่งด่วน,208.3k,0.55,17.3,0.1094,30,70,18 - 55+
5,17/11/66,Tiktok,Dr.Ice.Official,10005.0,วิธีกระชับรูขุมขนที่หมอแนะนำ,23.2k,0.41,14.0,0.1748,26,73,18 - 55+


**Preprocessing:** Tokenize the text, remove stopwords and spaces, and create a bag-of-words representation.

In [41]:
# Tokenize the 'Name' column
df['Tokenized_Name'] = df['Name'].apply(pythainlp.word_tokenize)
# df.head()
# Define a set of Thai stopwords and remove them along with spaces
stopwords = set(thai_stopwords())
filtered_tokens = [
    token for tokens in df['Tokenized_Name'] for token in tokens
    if token not in stopwords and token != ' '
]
# # Flatten the list of tokens and count word frequencies
# all_tokens = [token for tokens in df['Tokenized_Name'] for token in tokens]
word_freq = Counter(filtered_tokens)

# Display the most common words
print(word_freq.most_common(10))

[('กิน', 21), ('หมอ', 21), ('ทำ', 15), ('รีวิว', 12), ('ไหม', 12), ('ตัว', 11), ('ผิว', 11), ('หน้า', 11), ('สิว', 10), ('แนะนำ', 9)]


In [48]:
df['Tokenized_Name'] = df['Name'].apply(
    lambda x: [token for token in pythainlp.word_tokenize(x) if token not in stopwords and token != ' ']
)

# Convert the tokenized names back to a single string for vectorization
df['Processed_Name'] = df['Tokenized_Name'].apply(lambda x: ' '.join(x))

# Use CountVectorizer to create a bag-of-words model
# Vectorization: The CountVectorizer converts the processed text into a matrix of word frequencies.
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Processed_Name'])



Ensure 'No_Viewer' is in the correct format

In [49]:

def convert_viewers(viewers):
    if isinstance(viewers, (int, float)):
        return viewers
    if pd.isna(viewers):
        return None
    if 'M' in viewers:
        return float(viewers.replace('M', '')) * 1_000_000
    if 'k' in viewers:
        return float(viewers.replace('k', '')) * 1_000
    return float(viewers)

df['No_Viewer'] = df['No_Viewer'].apply(convert_viewers)
df['No_Viewer'].values.tolist()[:10]

[1100000.0,
 8761.0,
 825400.0,
 208300.0,
 23200.0,
 289100.0,
 1000000.0,
 1900000.0,
 815800.0,
 8347.0]

**Feature Engineering:** Convert the text data into numerical features (using the frequency of each word).

In [50]:
# Prepare the target variable
y = df['No_Viewer'].values

**Modeling:** Use a regression model to predict the *No_Viewer* count based on the word frequencies.

Regression Model: A linear regression model is used to find the relationship between the word frequencies and the No_Viewer count.

In [51]:
# Fit a linear regression model
# Regression Model: A linear regression model is used to find the relationship between
# the word frequencies and the No_Viewer count.
model = LinearRegression()
model.fit(X, y)

# Get the coefficients of the model
coefficients = model.coef_
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame to show the importance of each word
importance = pd.DataFrame({
    'Word': feature_names,
    'Coefficient': coefficients
})

# Sort by the absolute value of the coefficients to see which words have the most impact
importance['Absolute_Coefficient'] = importance['Coefficient'].abs()
importance_sorted = importance.sort_values(by='Absolute_Coefficient', ascending=False)

# Display the top 10 words impacting the 'No_Viewer' count
print(importance_sorted.head(10))

         Word   Coefficient  Absolute_Coefficient
54       olay  1.668393e+06          1.668393e+06
3          24  1.668393e+06          1.668393e+06
58    retinol  1.500168e+06          1.500168e+06
170        มก  1.122044e+06          1.122044e+06
171       มขน -1.116056e+06          1.116056e+06
129        ทา -1.106288e+06          1.106288e+06
102  คอลลาเจน  1.067405e+06          1.067405e+06
297     แนะนำ  1.028809e+06          1.028809e+06
315       โฟม  9.925962e+05          9.925962e+05
161        ผล  7.955966e+05          7.955966e+05


**Feature Importance:** The coefficients of the linear regression model indicate the impact of each word on the *No_Viewer* count. The higher the absolute value of a coefficient, the greater the impact.

In [74]:
!git branch

In [71]:
!git branch -M main

In [70]:
!git push --set-upstream main main

error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/peeti-sriwongsanguan/DrICE-TK.git'


In [55]:
!git checkout -b main

Switched to a new branch 'main'


In [56]:
!git status

On branch main

No commits yet

nothing to commit (create/copy files and use "git add" to track)


In [61]:
ls -a .git/

branches/  config  description  HEAD  hooks/  info/  objects/  refs/


In [66]:
!git push -u main main

error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/peeti-sriwongsanguan/DrICE-TK.git'


In [86]:
ls -a .git/

branches/  COMMIT_EDITMSG  config  description  HEAD  hooks/  index  info/  objects/  refs/
